In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import requests

In [3]:
def fix_random_seed(seed):
    try:
        np.random.seed(seed)
    except NameError:
        print("Warning: Numpy is not imported. Setting the seed for Numpy failed.")
    try:
        tf.random.set_seed(seed)
    except NameError:
        print("Warning: TensorFlow is not imported. Setting the seed for TensorFlow failed.")
    try:
        random.seed(seed)
    except NameError:
        print("Warning: random module is not imported. Setting the seed for random failed.")
 
fix_random_seed(9919)

In [4]:
iris = pd.read_csv('./iris/iris.data', header = None)
iris.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
iris.columns = ['sepal_length', 'sepal_width', 'petal_width', 'petal_length', 'label']
iris.head()

,sepal_length,sepal_width,petal_width,petal_length,label
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [6]:
iris['label'] = iris['label'].map({'Iris-setosa':0, 'Iris-versicolor':1, 'Iris-virginica':2})
iris.head()

,sepal_length,sepal_width,petal_width,petal_length,label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [7]:
iris = iris.sample(frac = 1.0 , random_state=9919)
iris.head()

,sepal_length,sepal_width,petal_width,petal_length,label
36,5.5,3.5,1.3,0.2,0
19,5.1,3.8,1.5,0.3,0
114,5.8,2.8,5.1,2.4,2
118,7.7,2.6,6.9,2.3,2
96,5.7,2.9,4.2,1.3,1


In [8]:
x = iris.iloc[:,:4]
x = x - x.mean(axis = 0)
x
y = tf.one_hot(iris.iloc[:,4] , depth = 3)
print(f'{x=}, {y=}')

x=     sepal_length  sepal_width  petal_width  petal_length
36      -0.343333        0.446    -2.458667     -0.998667
19      -0.743333        0.746    -2.258667     -0.898667
114     -0.043333       -0.254     1.341333      1.201333
118      1.856667       -0.454     3.141333      1.101333
96      -0.143333       -0.154     0.441333      0.101333
..            ...          ...          ...           ...
112      0.956667       -0.054     1.741333      0.901333
138      0.156667       -0.054     1.041333      0.601333
54       0.656667       -0.254     0.841333      0.301333
103      0.456667       -0.154     1.841333      0.601333
77       0.856667       -0.054     1.241333      0.501333

[150 rows x 4 columns], y=<tf.Tensor: shape=(150, 3), dtype=float32, numpy=
array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0.,

In [9]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

tf.keras.backend.clear_session()

In [10]:
model = Sequential([
    Dense(32, activation = 'relu' , input_shape= [4,]),
    Dense(16, activation = 'relu'),
    Dense(3 , activation = 'softmax')
])

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam' , metrics=['acc'] )
model.summary()

/Users/sanketmishra/Desktop/Desktop/college/PhD/.conda/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 739 (2.89 KB)

 Trainable params: 739 (2.89 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.fit(x, y , batch_size = 64 , epochs = 25)

Epoch 1/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.1693 - loss: 1.2533  
Epoch 2/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 901us/step - acc: 0.1732 - loss: 1.2010
Epoch 3/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 996us/step - acc: 0.2016 - loss: 1.1519
Epoch 4/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 976us/step - acc: 0.3157 - loss: 1.1059
Epoch 5/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 982us/step - acc: 0.6111 - loss: 1.0626
Epoch 6/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.6487 - loss: 1.0227
Epoch 7/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 939us/step - acc: 0.6632 - loss: 0.9869
Epoch 8/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 885us/step - acc: 0.6785 - loss: 0.9538
Epoch 9/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 837us/step - acc: 0.6640 - loss: 0.9221
Epoch 10/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 799us/step - acc: 0.6785 - loss: 0.8916
Epoch 11/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 893us/step - acc: 0.6838 - loss: 0.8629
Epoch 12/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.6838 - loss: 0.8357
Epoch 13/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 967us/step - 

In [35]:
from tensorflow.keras.layers import Input , Dense , Concatenate
from tensorflow.keras.models import Model
tf.keras.backend.clear_session()

In [36]:
inp1 = Input(shape = (4,))
inp2 = Input(shape = (2,))
out1 = Dense(16, activation = 'relu')(inp1)
out2 = Dense(16, activation = 'relu')(inp2)

In [37]:
out = Concatenate(axis = 1)([out1 , out2])
out = Dense(16 , activation = 'relu')(out)
out = Dense(3 , activation = 'softmax')(out)

In [38]:
model = Model(inputs = [inp1 , inp2] , outputs = [out])
model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['acc'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 2)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 16)        │         80 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 16)        │         48 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 32)        │          0 │ dense[0][0],      │
│ (Concatenate)       │                   │            │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 16)        │        528 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 3)         │         51 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 707 (2.76 KB)

 Trainable params: 707 (2.76 KB)

 Non-trainable params: 0 (0.00 B)

In [39]:
tf.keras.utils.plot_model(model)

You must install pydot (`pip install pydot`) for `plot_model` to work.


PCA


In [40]:
from sklearn.decomposition import PCA
pmodel = PCA(n_components = 2 , random_state = 9919)
x_pca = pmodel.fit_transform(x)

In [41]:
model.fit([x,x_pca],y, batch_size=64 , epochs = 25)

Epoch 1/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.3491 - loss: 1.0015  
Epoch 2/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 988us/step - acc: 0.3975 - loss: 0.9350
Epoch 3/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.5392 - loss: 0.8772
Epoch 4/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 976us/step - acc: 0.6693 - loss: 0.8276
Epoch 5/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.6693 - loss: 0.7849
Epoch 6/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 998us/step - acc: 0.6693 - loss: 0.7481
Epoch 7/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 945us/step - acc: 0.6693 - loss: 0.7162
Epoch 8/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 909us/step - acc: 0.6693 - loss: 0.6884
Epoch 9/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 878us/step - acc: 0.6693 - loss: 0.6640
Epoch 10/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 993us/step - acc: 0.6693 - loss: 0.6423
Epoch 11/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.6785 - loss: 0.6229
Epoch 12/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.6838 - loss: 0.6054
Epoch 13/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 991us/step - acc:

In [42]:
class MulBDense(tf.keras.layers.Layer):
    def __init__(self, units = 32 , input_dim = 32 , activation = None):
        super(MulBDense, self).__init__()
        self.units = units
        self.input_dim = input_dim
        self.activation = activation
    
    def build(self, input_shape):
        self.w = self.add_weight(shape = (input_shape[-1] , self.units) , initializer = 'glorot_uniform' , trainable = True)
        self.b = self.add_weight(shape = (self.units,) , initializer = 'glorot_uniform' , trainable = True)
        self.bmul = self.add_weight(shape = (self.units, ) , initializer = 'glorot_uniform' , trainable = True)
    
    def call(self , inputs):
        out = (tf.matmul(inputs , self.w) + self.b)* self.bmul
        return tf.keras.layers.Activation(self.activation)(out)

In [45]:
from tensorflow.keras.layers import Input , Dense , Concatenate
from tensorflow.keras.models import Model
tf.keras.backend.clear_session()
from sklearn.decomposition import PCA

In [50]:
inp = Input(shape = (4,))
out = Dense(16, activation='relu')(inp)

In [51]:
out = MulBDense(16, activation = 'relu')(out)
out = MulBDense(3 , activation = 'softmax')(out)

In [53]:
model = Model(inputs = [inp] , outputs = [out])
model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['acc'])
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 16)             │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mul_b_dense (MulBDense)         │ (None, 16)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mul_b_dense_1 (MulBDense)       │ (None, 3)              │            54 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 422 (1.65 KB)

 Trainable params: 422 (1.65 KB)

 Non-trainable params: 0 (0.00 B)

In [54]:
model.fit(x, y , batch_size = 64 , epochs = 25)

Epoch 1/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - acc: 0.3385 - loss: 1.0904  
Epoch 2/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.3385 - loss: 1.0888
Epoch 3/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.3385 - loss: 1.0872
Epoch 4/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.3385 - loss: 1.0855
Epoch 5/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.3385 - loss: 1.0837 
Epoch 6/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.3385 - loss: 1.0818
Epoch 7/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.3385 - loss: 1.0798
Epoch 8/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.3385 - loss: 1.0776
Epoch 9/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.3385 - loss: 1.0753
Epoch 10/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.3385 - loss: 1.0728
Epoch 11/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 972us/step - acc: 0.3385 - loss: 1.0701
Epoch 12/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 986us/step - acc: 0.3385 - loss: 1.0671
Epoch 13/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - acc: 0.3385 - l

Data


In [89]:
import os
data_dir = os.path.join('data','flower_images', 'flower_images')+ os.path.sep
csvds = tf.data.experimental.CsvDataset(
    os.path.join(data_dir, 'flower_labels.csv') , record_defaults=("", -1), header = True)
print(os.path.join(data_dir, 'flower_labels.csv'))
for item in csvds.take(5):
    print(item)

data/flower_images/flower_images/flower_labels.csv
(<tf.Tensor: shape=(), dtype=string, numpy=b'0001.png'>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'0002.png'>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'0003.png'>, <tf.Tensor: shape=(), dtype=int32, numpy=2>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'0004.png'>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'0005.png'>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)


2024-08-28 10:12:13.420317: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [106]:
fname_ds = csvds.map(lambda x, y : x)
label_ds = csvds.map(lambda x, y : y)
for item in fname_ds.take(5):
    print(item)

tf.Tensor(b'0001.png', shape=(), dtype=string)
tf.Tensor(b'0002.png', shape=(), dtype=string)
tf.Tensor(b'0003.png', shape=(), dtype=string)
tf.Tensor(b'0004.png', shape=(), dtype=string)
tf.Tensor(b'0005.png', shape=(), dtype=string)


In [95]:
def get_image(fpath):
    #load image from disk:
    img = tf.io.read_file(data_dir + fpath)
    #convert the string to a 3d tensor:
    img = tf.image.decode_png(img , channels=3)
    #convert dtype to floats:
    img = tf.image.convert_image_dtype(img , tf.float32)
    #resize and return
    return tf.image.resize(img , [64,64])

In [102]:
imageds = fname_ds.map(get_image)
labelds = label_ds.map(lambda x: tf.one_hot(x, depth = 10))
for item in tf.data.Dataset.zip((imageds , labelds)).take(5):
    print(item)

(<tf.Tensor: shape=(64, 64, 3), dtype=float32, numpy=
array([[[0.05490196, 0.0872549 , 0.0372549 ],
        [0.06764706, 0.09705883, 0.04411765],
        [0.06862745, 0.09901962, 0.04509804],
        ...,
        [0.09019609, 0.11862746, 0.0627451 ],
        [0.42941177, 0.5088235 , 0.26372552],
        [0.53333336, 0.6039216 , 0.34607846]],

       [[0.07352942, 0.10980393, 0.04705883],
        [0.0754902 , 0.11176471, 0.04901961],
        [0.07647059, 0.10882354, 0.05294118],
        ...,
        [0.0254902 , 0.02352941, 0.01764706],
        [0.05784314, 0.08333334, 0.0372549 ],
        [0.35294122, 0.44411767, 0.22352943]],

       [[0.18529412, 0.2676471 , 0.07941177],
        [0.12843138, 0.20000002, 0.06176471],
        [0.0872549 , 0.13039216, 0.05392157],
        ...,
        [0.02058824, 0.02254902, 0.01666667],
        [0.02450981, 0.02941177, 0.02254902],
        [0.03627452, 0.04607844, 0.04215687]],

       ...,

       [[0.36764705, 0.3009804 , 0.22549021],
        [0.341

In [104]:
datads = tf.data.Dataset.zip((imageds, labelds))
for item in datads:
    print(item)

(<tf.Tensor: shape=(64, 64, 3), dtype=float32, numpy=
array([[[0.05490196, 0.0872549 , 0.0372549 ],
        [0.06764706, 0.09705883, 0.04411765],
        [0.06862745, 0.09901962, 0.04509804],
        ...,
        [0.09019609, 0.11862746, 0.0627451 ],
        [0.42941177, 0.5088235 , 0.26372552],
        [0.53333336, 0.6039216 , 0.34607846]],

       [[0.07352942, 0.10980393, 0.04705883],
        [0.0754902 , 0.11176471, 0.04901961],
        [0.07647059, 0.10882354, 0.05294118],
        ...,
        [0.0254902 , 0.02352941, 0.01764706],
        [0.05784314, 0.08333334, 0.0372549 ],
        [0.35294122, 0.44411767, 0.22352943]],

       [[0.18529412, 0.2676471 , 0.07941177],
        [0.12843138, 0.20000002, 0.06176471],
        [0.0872549 , 0.13039216, 0.05392157],
        ...,
        [0.02058824, 0.02254902, 0.01666667],
        [0.02450981, 0.02941177, 0.02254902],
        [0.03627452, 0.04607844, 0.04215687]],

       ...,

       [[0.36764705, 0.3009804 , 0.22549021],
        [0.341

In [109]:
import tensorflow_datasets as tfds
tfds.list_builders()

2024-08-28 11:08:21.366552: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


['abstract_reasoning',
 'accentdb',
 'aeslc',
 'aflw2k3d',
 'ag_news_subset',
 'ai2_arc',
 'ai2_arc_with_ir',
 'aloha_mobile',
 'amazon_us_reviews',
 'anli',
 'answer_equivalence',
 'arc',
 'asqa',
 'asset',
 'assin2',
 'asu_table_top_converted_externally_to_rlds',
 'austin_buds_dataset_converted_externally_to_rlds',
 'austin_sailor_dataset_converted_externally_to_rlds',
 'austin_sirius_dataset_converted_externally_to_rlds',
 'bair_robot_pushing_small',
 'bc_z',
 'bccd',
 'beans',
 'bee_dataset',
 'beir',
 'berkeley_autolab_ur5',
 'berkeley_cable_routing',
 'berkeley_fanuc_manipulation',
 'berkeley_gnm_cory_hall',
 'berkeley_gnm_recon',
 'berkeley_gnm_sac_son',
 'berkeley_mvp_converted_externally_to_rlds',
 'berkeley_rpt_converted_externally_to_rlds',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'ble_wind_field',
 'blimp',
 'booksum',
 'bool_q',
 'bot_adversarial_dialogue',
 'bridge',
 'bridge_data_msr',
 'bucc',
 'c4',
 'c4_wsrs',
 'caltech10